# Deprecated - Create the graph for server version 4.0

This script should do the same as the database generating script in root. There is a change in how the database handles indices and constrains that is started updated here. However, we chose not to upgrade as 4.0 was pretty fresh as this project was developed.

In [2]:
from neo4j import GraphDatabase, basic_auth
from dotenv import load_dotenv
import os

# Does not quiiite work on 4.0 yet. Have to wait for 4.0 python driver, meantime I set encrypted false.

load_dotenv()

neo4jUser = os.getenv("NEO4J_USER")
neo4jPwd = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver("bolt://localhost:7687",auth=basic_auth(neo4jUser, neo4jPwd), encrypted = False)

AuthError: The client is unauthorized due to authentication failure.

In [20]:
def get_count_of_type(label, session):
    q = "MATCH (n:%s) RETURN count(n)" % label
    res = session.run(q)
    print("Added", res.value()[0], "nodes of type", label)
    
def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/cells")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [21]:
### CELLS

In [30]:
# Deleting all cell data in database
with driver.session() as session:
    cons_query = '''
        DROP CONSTRAINT ConsCellTypeId;
        DROP CONSTRAINT ConsCellClassId;
        DROP CONSTRAINT ConsCellGroupId;
        DROP CONSTRAINT ConsCellPhenotype;
        DROP CONSTRAINT ConsCellPhenotypeCategory;
        DROP CONSTRAINT ConsCellDescription;
        DROP CONSTRAINT ConsNeuralStructure;
        DROP INDEX IXNameCellType;
        DROP INDEX IXNameCellClass;
        DROP INDEX IXNameCellGroup;
        DROP INDEX IXNameCellPhenotype;
        DROP INDEX IXNameCellPhenotypeCategory;
        DROP INDEX IXNameNeuralStructure;
    '''
    for q in cons_query.split(";"):
        try:
            session.run(q)
        except:
            pass # Will fail when indices and constraints does not exist
    
    session.run('''
        MATCH (n)
        WHERE  n:CellType 
            OR n:CellClass
            OR n:CellGroup
            OR n:CellPhenotype 
            OR n:CellPhenotypeCategory 
            OR n:CellularRegion
            OR n:CellDescription
            OR n:NeuralStructure
        DETACH DELETE n
    ''')
        
print("wipeddatabase cells")

DatabaseError: Unable to drop constraint `ConsCellTypeId`: No such constraint ConsCellTypeId.

In [16]:
# Adding CellType to graph from cell_types.csv

csv_file_path = "file:///%s" % get_csv_path("cell_types.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellType { id: row.ID, name: row.Cell_type_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run("CREATE INDEX IXNameCellType FOR (n:CellType) ON (n.name)")
    session.run("CREATE CONSTRAINT ConsCellTypeId ON (n:CellType) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("CellType", session)

Added 105 nodes of type CellType


In [ ]:
# Adding CellClass to graph from cell_types.csv

csv_file_path = "file:///%s" % get_csv_path("cell_classes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellClass { id: row.ID, name: row.Cell_class_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run("CREATE INDEX ON :CellClass(name)")
    session.run("CREATE CONSTRAINT ON (n:CellClass) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("CellClass", session)

In [ ]:
# Adding CellGroup to graph from cell_types.csv

csv_file_path = "file:///%s" % get_csv_path("cell_groups.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellGroup { id: row.ID, name: row.Cell_group_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run("CREATE INDEX ON :CellGroup(name)")
    session.run("CREATE CONSTRAINT ON (n:CellGroup) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("CellGroup", session)

In [ ]:
# Adding CellPhenotype from cell_phenotypes.csv

csv_file_path = "file:///%s" % get_csv_path("cell_phenotypes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellPhenotype { id: row.ID, name: row.Phenotype, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:CellPhenotype) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :CellPhenotype(name)")
    session.run(query)
    get_count_of_type("CellPhenotype", session)

In [ ]:
# Adding CellPhenotypeCategory from cell_phenotype_categories.csv

csv_file_path = "file:///%s" % get_csv_path("cell_phenotype_categories.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellPhenotypeCategory { id:row.ID, name: row.Phenotype_category})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:CellPhenotype) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :CellPhenotypeCategory(name)")
    session.run(query)
    get_count_of_type("CellPhenotypeCategory", session)

In [ ]:
# Adding CellularRegion from cellular_regions.csv

csv_file_path = "file:///%s" % get_csv_path("cellular_regions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellularRegion { id:row.ID, name: row.Cellular_region, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:CellularRegion) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :CellularRegion(name)")
    session.run(query)
    get_count_of_type("CellularRegion", session)

In [ ]:
# Adding CellDescription from cell_descriptions.csv

csv_file_path = "file:///%s" % get_csv_path("cell_description.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellDescription { id: row.id, description: row.description, iri: row.iri})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:CellDescription) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("CellDescription", session)

In [ ]:
# Adding NeuralStructure from objects_of_intrest.csv

csv_file_path = "file:///%s" % get_csv_path("objects_of_interest.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:NeuralStructure { id:row.ID, name: row.Object_of_interest, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:NeuralStructure) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :NeuralStructure(name)")
    session.run(query)
    get_count_of_type("NeuralStructure", session)

In [ ]:
## Experiments

In [ ]:
def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/experiments")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [ ]:
# Deleting all cell data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Calculation
                OR n:StereologyDetail
                OR n:Distribution
                OR n:CellMorphology 
                OR n:Quantitation
                OR n:Analysis
                OR n:Experiment
                OR n:SectioningDetail
                OR n:SectioningInstrument
                OR n:ReporterIncubation
                OR n:Reporter
                OR n:ReporterLabel
                OR n:ReporterTarget
                OR n:ReporterType
                OR n:VisualizationProtocol
                OR n:Solution
                OR n:Software
                OR n:Microscope
                OR n:ElectronMicroscopeDetail
                OR n:LightFluorescenceMicroscopeDetail
            DETACH DELETE n
        ''')
        session.run('''
            DROP CONSTRAINT ON (n:Distribution) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:CellMorphology) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Quantitation) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Analysis) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Experiment) ASSERT n.id IS UNIQUE
            DROP INDEX ON :Cell(Distribution)
            DROP INDEX ON :CellType(CellMorphology)
            DROP INDEX ON :CellPhenotype(Quantitation)
            DROP INDEX ON :CellPhenotypeCategory(Analysis)
            DROP INDEX ON :NeuralStructure(Experiment)
        ''')
        print("wipeddatabase")

In [ ]:
# Adding Calculation to graph from calculations.csv
csv_file_path = "file:///%s" % get_csv_path("calculations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Calculation { id: row.ID, description: row.Calculation_performed})
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_type("Calculation", session)

In [ ]:
# Adding StereologyDetail to graph from stereology_details.csv
csv_file_path = "file:///%s" % get_csv_path("stereology_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:StereologyDetail { id: row.ID, name: row.Stereology_details_name, probe: row.Probe, identificationFeature: row.Identification_feature, disectorHeight: toInt(row.Disector_height), areaSubfraction: toFloat(row.Area_subfraction), heightSubfraction: toFloat(row.Height_subfraction), investigatedSections: toFloat(row.Investigated_sections), investigatedFields: toInt(row.Investigated_fields), countedObjects: toInt(row.Counted_objects), coefficientOfError: toFloat(row.Coefficient_of_error), estimatedVolume: toFloat(row.Estimated_volume), volumeUnit: row.Volume_unit, anyExceptProbe: toInt(row.any_except_probe)})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("StereologyDetail", session)

In [ ]:
# Adding Distribution to graph from distributions.csv
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Distribution { id: row.ID, name: row.Distribution_name, sectionSampling: row.Section_sampling, samplingFraction: row.Sampling_fraction, subsectionalSampling: row.Subsectional_sampling, finalEstimateBasis: row.Final_estimate_basis, distribution: row.Distribution, distributionDimensions: row.Distribution_dimensions, analysisTypePrimary: row.Analysis_type_primary, analysisTypeSecondary: row.Analysis_type_secondary})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Distribution(name)")
    session.run("CREATE CONSTRAINT ON (n:Distribution) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Distribution", session)

In [ ]:
# Adding CellMorphology to graph from cell_morphologies.csv
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:CellMorphology { id: row.ID, name: row.Morphology_name, neuromorphoId: row.Neuromorpho_ID, somaSurface: toFloat(row.Soma_surface), numberOfStems: toInt(row.Number_stems), numberOfBifurcations: toInt(row.Number_bifurcations), numberOfBranches: toInt(row.Number_branches), overallWidth: toFloat(row.Overall_width), overallHeight: toFloat(row.Overall_height), overallDepth: toFloat(row.Overall_depth), averageBranchDiameter: toFloat(row.Average_branch_diameter), averageContraction: toFloat(row.Average_contraction), totalArborLength: toFloat(row.Total_arbor_length), totalArborSurface: toFloat(row.Total_arbor_surface), totalArborVolume: toFloat(row.Total_arbor_volume), maxEuclideanDistance: toFloat(row.Max_Euclidean_distance), maxPathDistance: toFloat(row.Max_path_distance), maxBranchOrder: toInt(row.Max_branch_order), totalFragmentation: toInt(row.Total_fragmentation), partitionAsymmetry: toFloat(row.Partition_asymmetry), averageRalls: toInt(row.Average_Ralls), averageBifurcationAngleLocal: toFloat(row.Average_bifurcation_angle_local), averageBifurcationAngleRemote: toFloat(row.Average_bifurcation_angle_remote), fractalDimension: toFloat(row.Fractal_dimension), physicalIntegrity: row.Physical_integrity, structuralDomains: row.Structural_domains, morphologicalAttributes: row.Morphological_attributes, originalFormat: row.Original_format})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :CellMorphology(name)")
    session.run("CREATE CONSTRAINT ON (n:CellMorphology) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("CellMorphology", session)

In [ ]:
# Adding Quantitation to graph from cell_morphologies.csv
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Quantitation { id: row.ID, name: row.Quantitation_name, estimateRelevance: row.Estimate_relevance, sectionSampling: row.Section_sampling, samplingFraction: row.Sampling_fraction, subsectionalSampling: row.Subsectional_sampling, finalEstimateBasis: row.Final_estimate_basis, originalExtent: row.Original_extent, number: toInt(row.Number), numberSD: toInt(row.Number_SD), density: toFloat(row.Density), densityUnit: row.Density_unit, densitySD: toInt(row.Density_SD), volumetricDensity: toInt(row.Volumetric_density), estimateExtraction: row.Estimate_extraction})
    """ % csv_file_path

with driver.session() as session: 
    session.run("CREATE INDEX ON :Quantitation(name)")
    session.run("CREATE CONSTRAINT ON (n:Quantitation) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Quantitation", session)

In [ ]:
# Adding Analysis to graph from derived_data_records.csv
csv_file_path = "file:///%s" % get_csv_path("derived_data_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Analysis { id: row.ID, name: row.Derived_data_record_name, numberOfAnimals: row.Number_of_animals, dataType: row.Derived_data_type, shrinkageCorrection: row.Shrinkage_correction })
    """ % csv_file_path

with driver.session() as session:    
    session.run("CREATE INDEX ON :Analysis(name)")
    session.run("CREATE CONSTRAINT ON (n:Analysis) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Analysis", session)

In [ ]:
# Adding Experiment to graph from experiments.csv
csv_file_path = "file:///%s" % get_csv_path("experiments.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Experiment { id: row.ID, name: row.Experiment_name, animalStatus: row.Animal_status, ageLowerLimit: toInt(row.Age_lower_limit), ageUpperLimit: toInt(row.Age_upper_limit), weightLowerLimit: toInt(row.Weight_lower_limit), weightUpperLimit: toInt(row.Weight_upper_limit) })
    """ % csv_file_path

with driver.session() as session: 
    session.run("CREATE INDEX ON :Experiment(name)")
    session.run("CREATE CONSTRAINT ON (n:Experiment) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Experiment", session)

In [ ]:
# Adding SectioningDetail to graph from sectioning_details.csv
csv_file_path = "file:///%s" % get_csv_path("sectioning_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:SectioningDetail { id: row.ID, sectionThickness: toFloat(row.Section_thickness), sectionOrientation: row.Section_orientation })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("SectioningDetail", session)

In [ ]:
# Adding SectioningInstrument to graph from sectioning_instruments.csv
csv_file_path = "file:///%s" % get_csv_path("sectioning_instruments.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:SectioningInstrument { id: row.ID, name: row.Sectioning_instrument_name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("SectioningInstrument", session)

In [ ]:
# Adding ReporterIncubation to graph from reporter_incumations.csv
csv_file_path = "file:///%s" % get_csv_path("reporter_incubations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterIncubation { id: row.ID, order: row.Order, concentration: row.Concentration, time: toFloat(row.Time), temperature: toInt(row.temperature) })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterIncubation", session)

In [ ]:
# Adding Reporter to graph from reporters.csv
csv_file_path = "file:///%s" % get_csv_path("reporters.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Reporter { id: row.ID, name: row.Name, type: row.Reporter_type, uniqueId: row.Reporter_unique_ID, comment: row.Comment, originSpecie: row.Origin_species  })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Reporter", session)

In [ ]:
# Adding ReporterLabel to graph from reporter_labels.csv
csv_file_path = "file:///%s" % get_csv_path("reporter_labels.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterLabel { id: row.ID, name: row.Label})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterLabel", session)

In [ ]:
# Adding ReporterTarget to graph from reporter_targets.csv
csv_file_path = "file:///%s" % get_csv_path("reporter_targets.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ReporterTarget { id: row.ID, phenotype: row.Phenotype})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ReporterTarget", session)

In [ ]:
# Adding VisualizationProtocol to graph from visualization_protocols.csv
csv_file_path = "file:///%s" % get_csv_path("visualization_protocols.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:VisualizationProtocol { id: row.ID, name: row.Protocol_name, ontology: row.Ontological_identifier })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("VisualizationProtocol", session)

In [ ]:
# Adding Solution to graph from solutions.csv
csv_file_path = "file:///%s" % get_csv_path("solutions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Solution { id: row.ID, name: row.Solution_name })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Solution", session)

In [ ]:
# Adding Microscope to graph from microscopes.csv
csv_file_path = "file:///%s" % get_csv_path("microscopes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Microscope { id: row.ID, type: row.Microscope_type, ontology: row.Ontological_identifier })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Microscope", session)

In [ ]:
# Adding ElectronMicroscopeDetail to graph from electron_microscopy_details.csv
csv_file_path = "file:///%s" % get_csv_path("electron_microscopy_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ElectronMicroscopeDetail { id: row.ID, name: row.EM_details_name, gridType: row.Grid_type, magnification: row.Magnification })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("ElectronMicroscopeDetail", session)

In [ ]:
# Adding LightFluorescenceMicroscopeDetail to graph from light_fluorescence_microscopy_details.csv
csv_file_path = "file:///%s" % get_csv_path("light_fluorescence_microscopy_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:LightFluorescenceMicroscopeDetail { id: row.ID, name: row.Microscopy_details_name, mountingMedium: toInt(row.Mounting_medium), refractionMedium: row.Refraction_medium, numericalAperature: toFloat(row.Numerical_aperture), objectiveLens: row.Objective_lens, totalMagnification: row.Total_magnification, pixelSize: toFloat(row.Pixel_size), z_stack: toBoolean(row.Z_stack), opticalSliceSize: row.Optical_slice_size })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("LightFluorescenceMicroscopeDetail", session)

In [ ]:
# Adding Software to graph from softwares.csv
csv_file_path = "file:///%s" % get_csv_path("softwares.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Software { id: row.ID, name: row.Software_name, rrid: row.Software_RRID })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Software", session)

In [ ]:
## REGIONS

In [ ]:
def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/regions")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [ ]:
# Deleting all region data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:BrainRegion 
                OR n:RegionZone 
                OR n:RegionRecord 
                OR n:Nomenclature
                OR n:RegionOther
            DETACH DELETE n
            
        ''')
        session.run('''
            DROP CONSTRAINT ON (n:BrainRegion) ASSERT n.id IS UNIQUE
            DROP INDEX ON :BrainRegion(name)
            DROP CONSTRAINT ON (n:Nomenclature) ASSERT n.id IS UNIQUE
            DROP INDEX ON :Nomenclature(name)
            DROP CONSTRAINT ON (n:RegionOther) ASSERT n.id IS UNIQUE
            DROP INDEX ON :RegionOther(name)
        ''')
        print("wipeddatabase")

In [ ]:
# Adding regions to graph with BrainRegion label
csv_file_path = "file:///%s" % get_csv_path("regions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (d:BrainRegion {id: row.ID, name: row.Region_name, abbreviation: row.Abbreviation, comments: row.Comments})
    """ % csv_file_path

with driver.session() as session:    
    session.run("CREATE CONSTRAINT ON (n:BrainRegion) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :BrainRegion(name)")
    session.run(query)
    get_count_of_type("BrainRegion", session)

In [ ]:
# Adding region_records to graph with RegionRecord label
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:RegionRecord { id: row.ID, name: row.Region_record_name, coverage: row.Coverage, specificity: row.Specificity, numberOfOriginalRegions: toInt(row.No_original_regions), originalRegionRetained: toInt(row.Original_region_retained), parcellationScheme: row.Parcellation_scheme, atlasCoordinates: row.Atlas_coordinates, illustration: row.Illustration, semanticDescription: row.Semantic_description, annotatedImages: row.Annotated_images, regionalCharacteristics: row.Regional_characteristics, isAtlasRegion: row.Atlas_reg, serialSections: row.Serial_sections, collectorsComment: row.Collectors_comment, documentationScore: toInt(row.Documentation_score)})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("RegionRecord", session)

In [ ]:
# Adding region_zones to graph with RegionZone label
csv_file_path = "file:///%s" % get_csv_path("region_zones.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:RegionZone { id: row.ID, name: row.Region_zone, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("RegionZone", session)

In [ ]:
# Adding nomenclatures to graph with Nomenclature label
csv_file_path = "file:///%s" % get_csv_path("nomenclatures.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Nomenclature { id: row.ID, name: row.Nomenclature_name, version: row.Version, authors: row.Authors, published: toInt(row.Published), publication_type: row.Publication_type, doi: row.DOI, preferred: 1})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE CONSTRAINT ON (n:Nomenclature) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :Nomenclature(name)")
    session.run(query)
    get_count_of_type("Nomenclature", session)

In [ ]:
# Adding nomebclatures_other to graph with Nomenclature label
csv_file_path = "file:///%s" % get_csv_path("nomenclatures_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Nomenclature { id: row.ID, name: row.Nomenclature_name, version: row.Version, authors: row.Authors, published: toInt(row.Published), publicationType: row.Publication_type, preferred: 0 })
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Nomenclature", session)

In [ ]:
# Adding regions_other to graph with RegionOther label
csv_file_path = "file:///%s" % get_csv_path("regions_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (d:RegionOther {id: row.ID, name: row.Region_name, abbreviation: row.Abbreviation, comments: row.Comments})
    """ % csv_file_path

with driver.session() as session:    
    session.run("CREATE CONSTRAINT ON (n:RegionOther) ASSERT n.id IS UNIQUE")
    session.run("CREATE INDEX ON :RegionOther(name)")
    session.run(query)
    get_count_of_type("RegionOther", session)

In [ ]:
## SOURCES

In [ ]:
def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/sources")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [ ]:
# Deleting all source data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Source
                OR n:SourceOrigin
                OR n:ConsideredPaper
                OR n:ExclusionReason
            DETACH DELETE n
        ''')
        session.run('''
            DROP CONSTRAINT ON (n:Source) ASSERT n.id IS UNIQUE
            DROP INDEX ON :Source(Source)
        ''')
        print("wipeddatabase")

In [ ]:
# Adding Source to graph from sources.csv

csv_file_path = "file:///%s" % get_csv_path("sources.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Source {id: row.ID, title: row.Source_title, type: row.Source_type, insertedData: row.Inserted_date, publicationYear: toInt(row.Source_publication_year), sourceName: row.Source_name})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Source(name)")
    session.run("CREATE CONSTRAINT ON (n:Source) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Source", session)

In [ ]:
# Adding SourceOrigin to graph from sources.csv

csv_file_path = "file:///%s" % get_csv_path("source_origins_lookup.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:SourceOrigin {id: row.ID, name: row.Source_name, identifier: row.Identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("SourceOrigin", session)

In [ ]:
# Adding ConsideredPaper to graph from sources.csv
## TODO try to connect considered_paper and publication 
## based on first auth + year (published) with Source_name

csv_file_path = "file:///%s" % get_csv_path("considered_papers.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:ConsideredPaper {id: row.ID, title: row.Title, publishedYear: toInt(row.Published), firstAuthor: row.First_author, isIncluded: false})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    session.run("create index on :ConsideredPaper(id)")
    get_count_of_type("ConsideredPaper", session)
    
csv_file_path = "file:///%s" % get_csv_path("considered_papers_desicions.csv")

included_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        WITH row WHERE row.Decision =~ ".*(?i)Included.*"
        MATCH (n:ConsideredPaper {id: row.Paper})
        SET n.isIncluded = true
        RETURN COUNT(n)
    """ % csv_file_path

with driver.session() as session:    
    res = session.run(included_query)
    print("Number of considered papers that are included:",  res.value()[0])

In [ ]:
### Process considered paper decisions to find reasons where considered paper can get a relation to the reaseon.
## IF it doesnt have relation to excluded it  is included 

import pandas as pd
csv_file_path = "file:///%s" % get_csv_path("considered_papers_desicions.csv")
df = pd.read_csv(csv_file_path)

exclusion_reasons = df["Decision"].unique()

with driver.session() as session: 
    for reason in exclusion_reasons:
        if type(reason) == str and "included" not in reason.lower():

            q = "CREATE (:ExclusionReason {reason: '%s'})" % reason.replace("Excluded: ", "")
            session.run(q)
            
            # Relationship EXCLUSION_REASON between ConsideredPaper and ExclusionReason
            query="""
                    LOAD CSV WITH HEADERS FROM "%s" AS row
                    WITH row where row.Decision = '%s'
                    MATCH (a:ConsideredPaper { id: row.Paper})
                    MATCH (c:ExclusionReason { reason: '%s' })
                    MERGE (a)-[:EXCLUSION_REASON]->(c)
                """ % (csv_file_path, reason,  reason.replace("Excluded: ", ""))

            session.run(query)
    get_count_of_type("ExclusionReason", session)
    get_count_of_relationship("EXCLUSION_REASON", session)



In [ ]:
## SUBJECTS

In [ ]:
def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/subjects")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [ ]:
# Deleting all subject data in database
with driver.session() as session:
        session.run('''
            MATCH (n)
            WHERE  n:Specimen
                OR n:Specie 
                OR n:Strain 
                OR n:Substrain 
                OR n:Sex
                OR n:AgeCategory
                OR n:TransgenicLine
            DETACH DELETE n
        ''')
        session.run('''
            DROP CONSTRAINT ON (n:Specimen) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Specie) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Strain) ASSERT n.id IS UNIQUE
            DROP CONSTRAINT ON (n:Substrains) ASSERT n.id IS UNIQUE
            DROP INDEX ON :Specimen(name)
            DROP INDEX ON :Specie(name)
            DROP INDEX ON :Strain(name)
            DROP INDEX ON :Substrains(name)
        ''')
        print("wipeddatabase")

In [ ]:
# Adding Specimen to graph from specimens.csv

csv_file_path = "file:///%s" % get_csv_path("specimens.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Specimen {id: row.ID, name: row.Specimen_name, form: row.Specimen_form, order: row.Specimen_order})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Specimen(name)")
    session.run("CREATE CONSTRAINT ON (n:Specimen) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Specimen", session)

In [ ]:
# Adding Strain to graph from strains.csv

csv_file_path = "file:///%s" % get_csv_path("strains.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Strain {id: row.ID, name: row.Strain_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Strain(name)")
    session.run("CREATE CONSTRAINT ON (n:Strain) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Strain", session)

In [ ]:
# Adding Substrain to graph from substrains.csv

csv_file_path = "file:///%s" % get_csv_path("substrains.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Substrain {id: row.ID, name: row.Substrain_name, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Substrain(name)")
    session.run("CREATE CONSTRAINT ON (n:Substrain) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Substrain", session)

In [ ]:
# Adding Specie to graph from species.csv

csv_file_path = "file:///%s" % get_csv_path("species.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Specie {id: row.ID, name: row.Species, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:
    session.run("CREATE INDEX ON :Specie(name)")
    session.run("CREATE CONSTRAINT ON (n:Specie) ASSERT n.id IS UNIQUE")
    session.run(query)
    get_count_of_type("Specie", session)

In [ ]:
# Adding AgeCategory to graph from age_categories.csv

csv_file_path = "file:///%s" % get_csv_path("age_categories.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:AgeCategory {id: row.ID, name: row.Age_category, ontology: row.Ontological_identifier, description: row.Description})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("AgeCategory", session)

In [ ]:
# Adding Sex to graph from sex.csv

csv_file_path = "file:///%s" % get_csv_path("sex.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:Sex {id: row.ID, name: row.Sex, ontology: row.Ontological_identifier})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("Sex", session)

In [ ]:
# Adding TransgenicLine to graph from transgenic_lines.csv

csv_file_path = "file:///%s" % get_csv_path("transgenic_lines.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        CREATE (:TransgenicLine {id: row.ID, name: row.Transgenic_line_name, RRID: row.Transgenic_line_RRID})
    """ % csv_file_path

with driver.session() as session:    
    session.run(query)
    get_count_of_type("TransgenicLine", session)

In [ ]:
### CELL RELATIONSHIPS

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/cells")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [ ]:
# Relationship CELL_GROUP_MEMBERSHIP between CellClass and CellGroup

csv_file_path = "file:///%s" % get_csv_path("cell_classes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellClass { id: row.ID})
        MATCH (c:CellGroup { id: row.Cell_group_membership })
        MERGE (a)-[:CELL_GROUP_MEMBERSHIP]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELL_GROUP_MEMBERSHIP", session)

In [ ]:
# Relationship CELL_CLASS_MEMBERSHIP between CellType and CellClass

csv_file_path = "file:///%s" % get_csv_path("cell_types.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellType { id: row.ID})
        MATCH (c:CellClass { id: row.Cell_class_membership })
        MERGE (a)-[:CELL_CLASS_MEMBERSHIP]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELL_CLASS_MEMBERSHIP", session)

In [ ]:
# Relationship PHENOTYPE_CATEGORY between CellPhenotype and CellPhenotypeCategory

csv_file_path = "file:///%s" % get_csv_path("cell_phenotypes.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellPhenotype { id: row.ID})
        MATCH (c:CellPhenotypeCategory { id: row.Phenotype_category })
        MERGE (a)-[:PHENOTYPE_CATEGORY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PHENOTYPE_CATEGORY", session)

In [ ]:
# Relationship PHENOTYPE between Cell and CellPhenotype

csv_file_path = "file:///%s" % get_csv_path("celltype_phenotype.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellType { id: row.CellID})
        MATCH (c:CellPhenotype { id: row.PhenotypeID })
        MERGE (a)-[:PHENOTYPE]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PHENOTYPE", session)

In [ ]:
# Relationship CELL_DESCRIPTION between CellType, CellClass, CellGroup and CellDescription

csv_file_path = "file:///%s" % get_csv_path("cell_description.csv")
type_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellType { id: row.cell_type_id})
        MATCH (c:CellDescription { id: row.id })
        MERGE (a)-[:CELL_DESCRIPTION]->(c)
    """ % csv_file_path

class_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellClass { id: row.cell_class_id})
        MATCH (c:CellDescription { id: row.id })
        MERGE (a)-[:CELL_DESCRIPTION]->(c)
    """ % csv_file_path

group_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellGroup { id: row.cell_group_id})
        MATCH (c:CellDescription { id: row.id })
        MERGE (a)-[:CELL_DESCRIPTION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(type_query)
    session.run(class_query)
    session.run(group_query)
    get_count_of_relationship("CELL_DESCRIPTION", session)

In [ ]:
## EXPERIMENT RELATIONSHIPS

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/experiments")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

def get_count_of_relationship(rel_label, fromLabel, toLabel, session):
    q = "MATCH (:%s)-[r:%s]-(:%s) RETURN count(*)" % (fromLabel, rel_label, toLabel)
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", fromLabel, rel_label, toLabel)

In [ ]:
## Microscope RELATIONS

# :ElectronMicroscopeDetail - :MICROSCOPE_TYPE - :Microscope
csv_file_path = "file:///%s" % get_csv_path("electron_microscopy_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:ElectronMicroscopeDetail { id: row.ID})
        MATCH (c:Microscope { id: row.Microscope })
        MERGE (a)-[:MICROSCOPE_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE_TYPE", "ElectronMicroscopeDetail", "Microscope", session)
    
# :LightFluorescenceMicroscopeDetail - :MICROSCOPE_TYPE - :Microscope
csv_file_path = "file:///%s" % get_csv_path("light_fluorescence_microscopy_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:LightFluorescenceMicroscopeDetail { id: row.ID})
        MATCH (c:Microscope { id: row.Microscope })
        MERGE (a)-[:MICROSCOPE_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE_TYPE", "LightFluorescenceMicroscopeDetail", "Microscope", session)

# :Analysis - :MICROSCOPE - :ElectronMicroscopeDetail
csv_file_path = "file:///%s" % get_csv_path("derived_data_EMdetails.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:ElectronMicroscopeDetail { id: row.EMdetails })
        MERGE (a)-[:MICROSCOPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE", "Analysis", "ElectronMicroscopeDetail", session)

# :Analysis - :MICROSCOPE - :LightFluorescenceMicroscopeDetail
csv_file_path = "file:///%s" % get_csv_path("derived_data_LFMdetails.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record_ID})
        MATCH (c:LightFluorescenceMicroscopeDetail { id: row.LFMdetails_ID })
        MERGE (a)-[:MICROSCOPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("MICROSCOPE", "Analysis", "LightFluorescenceMicroscopeDetail", session)


In [ ]:
## Reporter RELATIONS
csv_file_path = "file:///%s" % get_csv_path("reporters.csv")

## :Reporter - :LABEL - :ReporterLabel
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Reporter { id: row.ID})
        MATCH (c:ReporterLabel { id: row.Label })
        MERGE (a)-[:LABEL]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("LABEL", "Reporter", "ReporterLabel", session)
    
## :Reporter - :TARGET - :ReporterTarget
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Reporter { id: row.ID})
        MATCH (c:ReporterTarget { id: row.Target })
        MERGE (a)-[:TARGET]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TARGET", "Reporter", "ReporterTarget", session)

In [ ]:
## SectioningDetail RELATIONS
csv_file_path = "file:///%s" % get_csv_path("sectioning_details.csv")

## :SectioningDetail - :SECTIONED_BY - :SectioningInstrument
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:SectioningDetail { id: row.ID})
        MATCH (c:SectioningInstrument { id: row.Sectioning_instrument })
        MERGE (a)-[:SECTIONED_BY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SECTIONED_BY", "SectioningDetail", "SectioningInstrument", session)


In [ ]:
## ReporterIncubation RELATIONS
csv_file_path = "file:///%s" % get_csv_path("reporter_incubations.csv")

## :ReporterIncubation - :REPORTER - :Reporter
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:ReporterIncubation { id: row.ID})
        MATCH (c:Reporter { id: row.Reporter })
        MERGE (a)-[:REPORTER]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REPORTER", "ReporterIncubation", "Reporter", session)

In [ ]:
### Experiment RELATIONS
csv_file_path = "file:///%s" % get_csv_path("experiments.csv")

## :Experiment - :ANAESTHETIC - :Solution
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Solution { id: row.Anaesthetic })
        MERGE (a)-[:ANAESTHETIC]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("ANAESTHETIC", "Experiment", "Solution", session)
    
## :Experiment - :PERFUSION_FIX_MEDIUM - :Solution
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Solution { id: row.Perfusion_fix_medium })
        MERGE (a)-[:PERFUSION_FIX_MEDIUM]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PERFUSION_FIX_MEDIUM", "Experiment", "Solution", session)

## :Experiment - :DATA_ORIGIN - :Source
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Experiment { id: row.ID})
        MATCH (c:Source { id: row.Source })
        MERGE (a)-[:DATA_ORIGIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_ORIGIN", "Experiment", "Source", session)


In [ ]:
### Analysis RELATIONS 
csv_file_path = "file:///%s" % get_csv_path("derived_data_records.csv")

## :Analysis - :SPECIMEN - :Specimen 
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:Specimen { id: row.Specimen })
        MERGE (a)-[:SPECIMEN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SPECIMEN", "Analysis", "Specimen", session)
    
## :Analysis - :OBJECT_OF_INTEREST - :RegionZone
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:NeuralStructure { id: row.Object_of_interest })
        MERGE (a)-[:OBJECT_OF_INTEREST]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("OBJECT_OF_INTEREST", "Analysis", "NeuralStructure", session)
    
## Add property to :Analysis - :OBJECT_OF_INTEREST - :RegionZone
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis)-[r:OBJECT_OF_INTEREST]-(b:NeuralStructure)
        WHERE row.OOI_recognition_criteria IS NOT NULL
        SET r.recognitionCriteria = row.OOI_recognition_criteria
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
## :Analysis - :CELL_TYPE_PUTATIVE - :Cell
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:CellType { id: row.Cell_type_putative })
        MERGE (a)-[:CELL_TYPE_PUTATIVE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELL_TYPE_PUTATIVE", "Analysis", "CellType", session)
    
## :Analysis - :VISUALIZATION_METHOD - :VisualizationProtocol
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.ID})
        MATCH (c:VisualizationProtocol { id: row.Visualization_method })
        MERGE (a)-[:VISUALIZATION_METHOD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("VISUALIZATION_METHOD", "Analysis", "VisualizationProtocol", session)


In [ ]:
### Quantitation RELATIONS

## :Analysis - :DATA_TYPE - :Quantitation
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:Quantitation { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "Quantitation", session)
    
## :Quantitation - :REGION_RECORD - :RegionRecord
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "Quantitation", "RegionRecord", session)
    
## :Quantitation - :REGION_ZONE - :RegionZone
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:RegionZone { id: row.Region_zone })
        MERGE (a)-[:REGION_ZONE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_ZONE", "Quantitation", "RegionZone", session)
    
## :Quantitation - :CELLULAR_TARGET_REGION - :CellularRegion
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:CellularRegion { id: row.Cellular_target_region })
        MERGE (a)-[:CELLULAR_TARGET_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELLULAR_TARGET_REGION", "Quantitation", "CellularRegion", session)
    
## :Quantitation - :TARGET_CELL - :Cell
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:CellType { id: row.Cellular_target_ID })
        MERGE (a)-[:TARGET_CELL]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TARGET_CELL", "Quantitation", "CellType", session)

## :Quantitation - :STEREOLOGY - :StereologyDetail
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:StereologyDetail { id: row.Stereology_details_record })
        MERGE (a)-[:STEREOLOGY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STEREOLOGY", "Quantitation", "StereologyDetail", session)
    
## :Quantitation - :SOFTWARE - :Software
csv_file_path = "file:///%s" % get_csv_path("quantitations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.ID})
        MATCH (c:Software { id: row.Software })
        MERGE (a)-[:SOFTWARE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SOFTWARE", "Quantitation", "Software", session)
    
## :Quantitation - :CALCULATION - :Calculation
csv_file_path = "file:///%s" % get_csv_path("calculations.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Quantitation { id: row.Quantitation_ID})
        MATCH (c:Calculation { id: row.ID })
        MERGE (a)-[:CALCULATION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CALCULATION", "Quantitation", "Calculation", session)

In [ ]:
### CellMorphology RELATIONS

## :Analysis - :DATA_TYPE - :CellMorphology
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:CellMorphology { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "CellMorphology", session)

## :CellMorphology - :REGION_RECORD - :RegionRecord
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "CellMorphology", "RegionRecord", session)
    
## :CellMorphology - :REGION_ZONE - :RegionZone
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:RegionZone { id: row.Region_zone })
        MERGE (a)-[:REGION_ZONE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_ZONE", "CellMorphology", "RegionZone", session)
    
## :CellMorphology - :RECONSTRUCTED_WITH - :Software
csv_file_path = "file:///%s" % get_csv_path("cell_morphologies.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:CellMorphology { id: row.ID})
        MATCH (c:Software { id: row.Reconstruction_method })
        MERGE (a)-[:RECONSTRUCTED_WITH]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("RECONSTRUCTED_WITH", "CellMorphology", "Software", session)

In [ ]:
### DISTRIBUTION RELATIONS

## :Analysis - :DATA_TYPE - :Distribution
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.Derived_data_record})
        MATCH (c:Distribution { id: row.ID })
        MERGE (a)-[:DATA_TYPE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("DATA_TYPE", "Analysis", "Distribution", session)

## :Distribution - :PERFORMED_ON - :Software    <- TODO FIGURE OUT WHY THESE ARE NOT ADDEDD
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:Software { id: row.Software })
        MERGE (a)-[:PERFORMED_ON]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PERFORMED_ON", "Distribution", "Software", session)
    
## :Distribution - :CELLULAR_REGION - :CellularRegion
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:CellularRegion { id: row.Cellular_region })
        MERGE (a)-[:CELLULAR_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
# Add property to :relation
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        WITH row WHERE row.Cellular_region_cytochemical_ID is not null
        MATCH (a:Distribution {id: row.ID})-[r:CELLULAR_REGION]-(b:CellularRegion)
        SET r.cytochemicalId = row.Cellular_region_cytochemical_ID
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELLULAR_REGION", "Distribution", "CellularRegion", session)

## :Distribution - :REGION_RECORD - :RegionRecord
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:RegionRecord { id: row.Region_record })
        MERGE (a)-[:REGION_RECORD]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_RECORD", "Distribution", "RegionRecord", session)

## :Distribution - :STEREOLOGY - :StereologyDetail
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:StereologyDetail { id: row.Stereology_details_record })
        MERGE (a)-[:STEREOLOGY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STEREOLOGY", "Distribution", "StereologyDetail", session)
    
## :Distribution - :RELATED_TO - :Quantitation
csv_file_path = "file:///%s" % get_csv_path("distributions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Distribution { id: row.ID})
        MATCH (c:Quantitation { id: row.Related_quantitation })
        MERGE (a)-[:RELATED_TO]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("RELATED_TO", "Distribution", "Quantitation", session)



In [ ]:
### Derived relations from Experiments and Analysis (DDR) and Specimen

## :Analysis - :SECTIONING_DETAIL - :SectioningDetail
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_sectioning_details.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.analysisId})
        MATCH (c:SectioningDetail { id: row.sectioningDetailsId })
        MERGE (a)-[:SECTIONING_DETAIL]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SECTIONING_DETAIL", "Analysis", "SectioningDetail", session)

    
## :Analysis - :REPORTER_INCUBATION - :ReporterIncubation
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_reporters.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.analysisId})
        MATCH (c:ReporterIncubation { id: row.reporterId })
        MERGE (a)-[:REPORTER_INCUBATION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REPORTER_INCUBATION", "Analysis", "ReporterIncubation", session)
    
## :Analysis - :PART_OF_EXPERIMENT - :Experiment
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_experiments.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Analysis { id: row.analysisId})
        MATCH (c:Experiment { id: row.experimentId })
        MERGE (a)-[:PART_OF_EXPERIMENT]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PART_OF_EXPERIMENT", "Analysis", "Experiment", session)



In [ ]:
## REGION RELATIONSHIPS

def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/regions")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [ ]:
# Relationship PRIMARY_REGION between BrainRegion and RegionRecord
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord { id: row.ID})
        MATCH (c:BrainRegion { id: row.Region})
        MERGE (a)-[:PRIMARY_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PRIMARY_REGION", session)

In [ ]:
# Relationship SECONDARY_REGION between BrainRegion and RegionRecord
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord { id: row.ID})
        MATCH (c:BrainRegion { id: row.Secondary_region })
        MERGE (a)-[:SECONDARY_REGION]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SECONDARY_REGION", session)

In [ ]:
# Relationship NOMENCLATURE_SPECIE between Nomenclature and Specie
csv_file_path = "file:///%s" % get_csv_path("nomenclatures.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:NOMENCLATURE_SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
csv_file_path = "file:///%s" % get_csv_path("nomenclatures_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:NOMENCLATURE_SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("NOMENCLATURE_SPECIE", session)

In [ ]:
# Relationship NOMENCLATURE_STRAIN between Nomenclature and Strain
csv_file_path = "file:///%s" % get_csv_path("nomenclatures.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Strain { id: row.Strain })
        MERGE (a)-[:NOMENCLATURE_STRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    
csv_file_path = "file:///%s" % get_csv_path("nomenclatures_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Nomenclature { id: row.ID})
        MATCH (c:Strain { id: row.Strain })
        MERGE (a)-[:NOMENCLATURE_STRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("NOMENCLATURE_STRAIN", session)

In [ ]:
# Relationship NAMING between All BrainRegion and Nomenclature
csv_file_path = "file:///%s" % get_csv_path("regions.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BrainRegion { id: row.ID})
        MATCH (c:Nomenclature { id: row.Nomenclature })
        MERGE (a)-[:NAMING]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("NAMING", session)
    
# Relationship REGION_OTHER_NAMING between All RegionOther and Nomenclature
csv_file_path = "file:///%s" % get_csv_path("regions_other.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionOther { id: row.ID})
        MATCH (c:Nomenclature { id: row.Nomenclature })
        MERGE (a)-[:REGION_OTHER_NAMING]->(c)
    """ % csv_file_path

csv_file_path = "file:///%s" % get_csv_path("original_region_records.csv")
rel_query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionOther {id: row.Original_region})-[r:REGION_OTHER_NAMING]-(b:Nomenclature)
        SET r.originalCoordinateMinAP = row.Original_coord_AP_min
        SET r.originalCoordinateMaxAP = row.Original_coord_AP_max
        SET r.originalCoordinateMinDV = row.Original_coord_DV_min
        SET r.originalCoordinateMaxDV = row.Original_coord_DV_max
        SET r.originalCoordinateMinML = row.Original_coord_ML_min
        SET r.originalCoordinateMaxML = row.Original_coord_ML_max
""" % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_OTHER_NAMING", session)

In [ ]:
# Relationship ORIGINAL_FRAMEWORK between Regions_records and Nomenclatures_other
csv_file_path = "file:///%s" % get_csv_path("region_records.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:RegionRecord { id: row.ID})
        MATCH (c:Nomenclature { id: row.Original_framework })
        MERGE (a)-[:ORIGINAL_FRAMEWORK]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("ORIGINAL_FRAMEWORK", session)

In [ ]:
### Add Hierarchical relations between regions
# RAT:
csv_file_path = "file:///%s" % get_csv_path("region_hierarchy_waxholm.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BrainRegion { id: row.ID})
        MATCH (c:BrainRegion { id: row.Part_of })
        MERGE (a)-[:PART_OF]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PART_OF", session)
    
# MOUSE:
csv_file_path = "file:///%s" % get_csv_path("region_hierarchy_amba.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:BrainRegion { id: row.ID})
        MATCH (c:BrainRegion { id: row.Part_of })
        MERGE (a)-[:PART_OF]->(c)
    """ % csv_file_path
with driver.session() as session:
    session.run(query)
    get_count_of_relationship("PART_OF", session)


In [ ]:
## SOURCES RELATIONSHIPS

def get_count_of_relationship(rel_label, fromLabel, toLabel, session):
    q = "MATCH (:%s)-[r:%s]-(:%s) RETURN count(*)" % (fromLabel, rel_label, toLabel)
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", fromLabel, rel_label, toLabel)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/sources")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [ ]:
# Relationship COLLECTED_FROM between Source and SourceOrigin
csv_file_path = "file:///%s" % get_csv_path("sources.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Source { id: row.ID})
        MATCH (c:SourceOrigin { id: row.Source_origin })
        MERGE (a)-[:COLLECTED_FROM]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("COLLECTED_FROM", "Source", "SourceOrigin", session)

In [ ]:
# Relationship COLLECTED_FROM between ConsideredPaper and SourceOrigin
csv_file_path = "file:///%s" % get_csv_path("considered_papers.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:ConsideredPaper { id: row.ID})
        MATCH (c:SourceOrigin { id: row.Journal })
        MERGE (a)-[:COLLECTED_FROM]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("COLLECTED_FROM", "ConsideredPaper", "SourceOrigin", session)

In [ ]:
## SUBJECT RELATIONSHIPS

def get_count_of_relationship(label, session):
    q = "MATCH ()-[r:%s]-() RETURN count(*)" %label
    res = session.run(q)
    print("Added", res.value()[0], "relationships of type", label)

def get_csv_path(csv_file):
    path_all_csv = os.path.realpath("Data/csvs/basal_ganglia/subjects")
    return os.path.join(path_all_csv, csv_file).replace("\\","/")

In [ ]:
# Relationship SPECIE between Specimen and Specie
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SPECIE", session)

In [ ]:
# Relationship STRAIN between Specimen and Strain
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Strain { id: row.Strain })
        MERGE (a)-[:STRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("STRAIN", session)

In [ ]:
# Relationship SUBSTRAIN between Specimen and Specie
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Substrain { id: row.Substrain })
        MERGE (a)-[:SUBSTRAIN]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SUBSTRAIN", session)

In [ ]:
# Relationship TRANSGENIC_LINE between Specimen and TransgenicLine
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:TransgenicLine { id: row.Transgenic_line_name })
        MERGE (a)-[:TRANSGENIC_LINE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("TRANSGENIC_LINE", session)

In [ ]:
# Relationship SEX between Specimen and Sex
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:Sex { id: row.Sex })
        MERGE (a)-[:SEX]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("SEX", session)

In [ ]:
# Relationship AGE_CATEGORY between Specimen and AgeCategory
csv_file_path = "file:///%s" % get_csv_path("derived_ddr_specimen.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Specimen { id: row.SpecimenId})
        MATCH (c:AgeCategory { id: row.Sex })
        MERGE (a)-[:AGE_CATEGORY]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("AGE_CATEGORY", session)

In [ ]:
# Relationship FROM_SPECIE between Strain and Specie
csv_file_path = "file:///%s" % get_csv_path("strains.csv")
query="""
        LOAD CSV WITH HEADERS FROM "%s" AS row
        MATCH (a:Strain { id: row.ID})
        MATCH (c:Specie { id: row.Species })
        MERGE (a)-[:FROM_SPECIE]->(c)
    """ % csv_file_path

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("FROM_SPECIE", session)


In [ ]:
### EXTENDED RELATIONSHIPS

In [ ]:
# Relationship CELLS_IN_REGION between BrainRegion and Cell

query="""
        MATCH (b:BrainRegion)<-[:PRIMARY_REGION]-(r:RegionRecord)<-[:REGION_RECORD]-(d)<-[:DATA_TYPE]-(:Analysis)-[:CELL_TYPE_PUTATIVE]->(c:CellType)
        MERGE (b)-[:CELLS_IN_REGION]->(c)
    """

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("CELLS_IN_REGION", session)

In [ ]:
# Relationship REGION_SPECIE between BrainRegion and Specie
query="""
        MATCH (a:BrainRegion)-[:NAMING]->(b:Nomenclature)-[:NOMENCLATURE_SPECIE]->(c:Specie)
        MERGE (a)-[:REGION_SPECIE]->(c)
    """

with driver.session() as session:
    session.run(query)
    get_count_of_relationship("REGION_SPECIE", session)